In [7]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_file_path : Path
    test_data_filepath : Path
    criterion : str
    max_depth : int

In [9]:
from regression.constants import constant
from regression.utils.common_func import read_yaml, create_dir, save_json
from regression import logger
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score

import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
from mlflow.models import infer_signature

In [10]:
class EvaluationConfigurationManager:
    def __init__(self,
                 config_filepath = constant.CONFIG_FILE_PATH,
                 param_filepath = constant.PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

    def get_evaluation_config(self) -> ModelEvaluationConfig:
        eval_config = ModelEvaluationConfig(
            model_file_path = self.config.prepare_model.model_file_path,
            test_data_filepath=self.config.data_split.test_data_filepath,
            criterion=self.params.criterion,
            max_depth=self.params.max_depth)
        return eval_config


In [11]:
class Evaluation:
    def __init__(self, config : ModelEvaluationConfig):
        self.config = config

    def load_model(self) ->str:
        try:
            logger.info(f'loading the model from {self.config.model_file_path}')
            self.model = joblib.load(self.config.model_file_path)
            logger.info("Model loaded Successfully")

        except Exception as e:
            logger.error(f"Error occurred while loading the model: {e}")
            raise e
        
    def evaluation(self) ->str:
        logger.info(f'loading test data into dataframe')
        test_df = pd.read_csv(self.config.test_data_filepath)
        y_test = test_df['quality']
        x_test = test_df.drop('quality', axis=1)
        y_pred = self.model.predict(x_test)
        self.score = accuracy_score(y_test, y_pred)
        self.rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        self.mae = mean_absolute_error(y_test, y_pred)
        self.r2 = r2_score(y_test, y_pred)

    def log_into_mlflow(self):
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            test_df = pd.read_csv(self.config.test_data_filepath)
            y_test = test_df['quality']
            x_test = test_df.drop('quality', axis=1)
            # y_pred = self.model.predict(x_test)
            mlflow.log_param("criterion", self.config.criterion)
            mlflow.log_param("max_depth", self.config.max_depth)

            mlflow.log_metric("RMSE", self.rmse)
            mlflow.log_metric("MAE", self.mae)
            mlflow.log_metric("R2", self.r2)

            signature = infer_signature(y_test, self.model.predict(x_test))

            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(self.model, "model", registered_model_name="DescisionClassifier")
            else:
                mlflow.sklearn.log_model(self.model, "model", signature=signature)




In [12]:
try:
    config_manager = EvaluationConfigurationManager()
    eval_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.load_model()
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e


[2024-09-14 07:20:35,824:INFO:common_func:yaml file: ..\config\config.yaml loaded successfully]
[2024-09-14 07:20:35,826:INFO:common_func:yaml file: ..\params.yaml loaded successfully]
[2024-09-14 07:20:35,826:INFO:1665163982:loading the model from ../artifacts/model/trained_model.h5]
[2024-09-14 07:20:35,826:INFO:1665163982:Model loaded Successfully]
[2024-09-14 07:20:35,830:INFO:1665163982:loading test data into dataframe]


d:\SONU\folder c\Desktop\Portfolio Github Projects\mlops-structure\.venv\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
